In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import GPy

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import ridge_regression
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

## Load Data

In [2]:
became_automated = pd.read_csv("../../../data/helpers/automation/delta_9_15.csv")
tt6 = pd.read_csv('../../../data/helpers/tt6.csv')
tt9 = pd.read_csv('../../../data/helpers/tt9.csv')
tt15 = pd.read_csv('../../../data/helpers/tt15.csv')
emerged_tech = pd.read_csv('../../../data/helpers/emerged_occs_and_tech/emerged_all.csv')

In [3]:
# Create X (feature matrix)

def create_factor_df(path, scale_id):
    X = pd.read_table(path)
    X = X[X['Scale ID'] == scale_id][['O*NET-SOC Code', 'Element Name', 'Data Value']].reset_index(drop = True)
    X = X.pivot(index = 'O*NET-SOC Code', columns = 'Element Name', values = 'Data Value')
    X.index.name = None
    X['O*NET-SOC Code'] = X.index
    return X

In [4]:
X = create_factor_df('../../../data/databases/db09/Skills.txt', 'LV')
np.linalg.cond(X.drop('O*NET-SOC Code', axis = 1))

81.320255227944003

> Note – strong multicollinearity

# Y = AUTOMATION % CHANGE

First, clean data and add y_target

In [5]:
X = X.merge(became_automated[['O*NET-SOC Code', 'delta_pct']], how = 'left', on = 'O*NET-SOC Code')
y = X['delta_pct']
X = X.drop('delta_pct', axis = 1)
X = X[y.notnull()]
y = pd.DataFrame(y[y.notnull()])

In [6]:
len(X), len(y)

(759, 759)

In [7]:
codes = X['O*NET-SOC Code']
X = X.reset_index(drop = True)
X = X.drop('O*NET-SOC Code', axis = 1)

In [12]:
became_automated.to_csv('../../../data/helpers/skills/automation_targets.csv', index = False)
X.to_csv('../../../data/helpers/skills/skills_2009.csv', index = False)
y.to_csv('../../../data/helpers/skills/automation_y.csv', index = False)
codes.to_csv('../../../data/helpers/skills/codes_index.csv', header = True, index = False)

Filter the data

In [10]:
# remove missing data
def filt_nan(X, y):
    filt = y.notnull()
    X = X[filt]
    y = y[filt]
    return X, y

X, y = filt_nan(X, y)

In [11]:
# focus only on data that has surpassed a certain threshold
def filt_changed(X, y, threshold):
    filt = abs(y) > threshold
    X = X[filt]
    y = y[filt]
    return X, y

X, y = filt_changed(X, y, 0.1)

Then split data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
data = [X_train, X_test, y_train, y_test]
X_train, X_test, y_train, y_test = map(lambda x: np.array(x), data)

Clean the data (GP regression requirements)

In [13]:
y_train, y_test = y_train[:,np.newaxis].squeeze(), y_test[:,np.newaxis].squeeze()

Then create reporting functions

In [14]:
def get_r_squared(y_pred, y_true):
    ssr = np.sum((y_pred - np.mean(y_true))**2)
    sst = np.sum((y_true - np.mean(y_true))**2)
    return float(ssr)/(sst)

def get_mse(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

def get_rmse(y_pred, y_true):
    return np.sqrt(get_mse(y_pred, y_true))

def score_model(y_pred, y_true):
    mean = np.mean(y_true)
    rmse = get_rmse(y_pred, y_true)
    r2 = get_r_squared(y_pred, y_true)
    (mean, rmse, r2) = tuple(map(lambda x: round(x, 5), [mean, rmse, r2]))
    
    s = """    MODEL RESULTS:
    --------------

    Mean  |  {}  |
    RMSE  |  {}  |
    R2    |  {}  |
    """.format(mean, rmse, r2)
    print s

Then run models

### Random Forest

In [15]:
rf = RandomForestRegressor(n_estimators = 1000)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
score_model(predictions, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  0.0076  |
    RMSE  |  0.29869  |
    R2    |  0.12423  |
    


### Gaussian Process

Create several different kernels:

In [16]:
# Create nearly all possible kernels
exp = GPy.kern.Exponential(input_dim = X.shape[1], ARD = True)
lin = GPy.kern.Linear(input_dim = X.shape[1], ARD = True)
expquad = GPy.kern.ExpQuad(input_dim = X.shape[1], ARD = True)
rbf = GPy.kern.RBF(input_dim = X.shape[1], ARD = True)
per = GPy.kern.PeriodicExponential(input_dim = X.shape[1], ARD = True)
exp_t_lin = exp * lin
lin_t_expquad = lin * expquad
per_t_lin = per * lin
per_t_exp = per * exp
exp_p_lin = exp + lin
lin_p_expquad = lin + expquad
per_p_lin = per + lin
per_p_exp = per + exp

# # list them for the optimize function
kernel_list = [exp]
#              lin,
#              expquad,
#              rbf,
#              per,
#              exp_t_lin,
#              lin_t_expquad,
#              per_t_lin,
#              per_t_exp,
#              exp_p_lin,
#              lin_p_expquad,
#              per_p_lin,
#              per_p_exp]

TypeError: __init__() got an unexpected keyword argument 'ARD'

In [ ]:
def optimize_gaussian(X_train, y_train, X_test, y_test, kernel_list):
    """
    Iterates through covariance functions and identifies the one
    with the lowest RMSE, and returns:
        - model,
        - predictions,
        - kernel name,
        - RMSE
    """
    best_model = None
    best_rmse = float('inf')
    best_name = None
    best_preds = None
    nowork = []
    
    new_ys = []
    for y_array in [y_train, y_test]:
        if len(y_array.shape) == 1:
            new_ys.append(y_array[:,np.newaxis])
    y_train, y_test = new_ys
    
    for kernel in kernel_list:
        try:
            m = GPy.models.GPRegression(X_train, y_train, kernel = kernel)
            m.optimize(max_f_eval = 1000)
            m.optimize_restarts(num_restarts=100)
            mean, var = m.predict(np.array(X_test))
            rmse = get_rmse(mean.squeeze(), y_test)
            if rmse < best_rmse:
                best_model = m
                best_rmse = rmse
                best_name = kernel.name
                best_preds = mean.squeeze()
            print """{} RMSE: {}
            """.format(kernel.name, rmse)
        except:
            nowork.append(kernel.name)

    return best_model, best_preds, best_name, best_rmse

In [63]:
m, y_pred, name, rmse = optimize_gaussian(X_train, y_train, X_test, y_test, kernel_list)

Optimization restart 1/100, f = 22.2591527264
Optimization restart 2/100, f = 19.8512849743
Optimization restart 3/100, f = 17.9501666646
Optimization restart 4/100, f = 22.1969231383
Optimization restart 5/100, f = 19.6988059537
Optimization restart 6/100, f = 19.9018524729
Optimization restart 7/100, f = 22.2516823665
Optimization restart 8/100, f = 22.4959553506
Optimization restart 9/100, f = 20.7927087769
Optimization restart 10/100, f = 20.1406099509
Optimization restart 11/100, f = 21.9037567589
Optimization restart 12/100, f = 22.2313138067
Optimization restart 13/100, f = 18.9497539869
Optimization restart 14/100, f = 20.8168735752
Optimization restart 15/100, f = 24.0454695698
Optimization restart 16/100, f = 21.440440557
Optimization restart 17/100, f = 17.8709674946
Optimization restart 18/100, f = 20.3790604468
Optimization restart 19/100, f = 20.0211958532
Optimization restart 20/100, f = 20.8754899295
Optimization restart 21/100, f = 24.3992879301
Optimization restart 22

In [59]:
m.kern.lengthscale

Index,GP_regression.Exponential.lengthscale,Constraint,Prior,Tied to
[0],680.778394747,+ve,,N/A
[1],1875.99510528,+ve,,N/A
[2],982.281911815,+ve,,N/A
[3],0.434098140273,+ve,,N/A
[4],1829.33483703,+ve,,N/A
[5],727.831523813,+ve,,N/A
[6],1321.0880452,+ve,,N/A
[7],2086.25969006,+ve,,N/A
[8],1196.09859412,+ve,,N/A
[9],3271.72753407,+ve,,N/A


In [15]:
score_model(y_pred, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  0.01241  |
    RMSE  |  0.26805  |
    R2    |  0.00215  |
    


### LASSO Regression

In [16]:
lasso = sklearn.linear_model.Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
score_model(y_pred, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  0.01241  |
    RMSE  |  0.26892  |
    R2    |  0.00864  |
    


# Y = AUTOMATION (abs) CHANGE

In [17]:
X = create_factor_df('../data/db09/Skills.txt', 'LV')
X = X.merge(became_automated[['O*NET-SOC Code', 'delta']], how = 'left', on = 'O*NET-SOC Code').drop('O*NET-SOC Code', axis = 1)
y = X['delta']
X = X.drop('delta', axis = 1)
X = X[y.notnull()]
y = y[y.notnull()]

X, y = np.array(X), np.array(y).squeeze()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Random Forests

In [19]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
score_model(y_pred, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  -0.02247  |
    RMSE  |  0.32864  |
    R2    |  0.20366  |
    


### Gaussian Process

In [18]:
m, y_pred, name, rmse = optimize_gaussian(X_train, y_train, X_test, y_test, kernel_list)

NameError: name 'kernel_list' is not defined

In [225]:
score_model(y_pred, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  0.021  |
    RMSE  |  0.33218  |
    R2    |  0.00599  |
    


# Y = log(\# adopted technologies)

In [82]:
adoptions = pd.read_csv('tech_adoptions.csv')
num_adoptions = pd.DataFrame(adoptions.groupby('O*NET-SOC Code')['Title'].count()).reset_index()
num_adoptions.columns = ['O*NET-SOC Code', 'num_adopted']


X = create_factor_df('../data/db09/Skills.txt', 'LV')
X = X.merge(num_adoptions[['O*NET-SOC Code', 'num_adopted']], how = 'left', on = 'O*NET-SOC Code').drop('O*NET-SOC Code', axis = 1)
X.num_it_adopted = X.num_adopted.fillna(value = 0)
y = X.num_adopted
X = X.drop('num_adopted', axis = 1)
X = X[y.notnull()]
y = y[y.notnull()]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
m, y_pred, name, rmse = optimize_gaussian(X_train, y_train, X_test, y_test, kernel_list)
score_model(y_pred, y_test)

Exponential RMSE: 43.9328967308
            
linear RMSE: 43.9328967177
            
ExpQuad RMSE: 43.9328967303
            
rbf RMSE: 43.9328967303
            
periodic_exponential RMSE: 43.9328967165
            
    MODEL RESULTS:
    --------------

    Mean  |  28.07602  |
    RMSE  |  43.9329  |
    R2    |  0.69035  |
    


> ### This appears to explain a lot of variation; pursue feature importances with Random Forest

In [86]:
rf = RandomForestRegressor(n_estimators = 200)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
score_model(y_pred, y_test)

    MODEL RESULTS:
    --------------

    Mean  |  28.07602  |
    RMSE  |  30.14601  |
    R2    |  0.31317  |
    


In [102]:
feature_importances = zip(X_train.columns, rf.feature_importances_)
feature_importances.sort(key=lambda x: x[1])
top_ten_most_important = [i[0] for i in feature_importances[:10]]
for feature in top_ten_most_important:
    vals  = X[feature]
    print feature," \t \t ", np.corrcoef(np.array(vals), np.array(y).squeeze())[0][1]

Monitoring  	 	  0.0545767269121
Operations Analysis  	 	  0.114527258596
Complex Problem Solving  	 	  0.0784093588529
Active Listening  	 	  -0.0200878790641
Active Learning  	 	  0.112961857603
Systems Analysis  	 	  0.0184033406038
Critical Thinking  	 	  0.0889422907895
Negotiation  	 	  -0.0477685857588
Persuasion  	 	  -0.00337846695395
Management of Material Resources  	 	  0.12839245353


> ###So it seems the most important features have low correlation! There is some *mixture*
> ### of features that is then important. So:
    > ### * What is a better measure of feature importances
    > ### * How do you find great combinations of variables?

# Y = log(\# adopted IT technologies)

In [80]:
just_it = adoptions[adoptions.segment_title == 'Information Technology Broadcasting and Telecommunications']
it_adoptions = pd.DataFrame(just_it.groupby("O*NET-SOC Code").Title.count()).reset_index()
it_adoptions.columns = ['O*NET-SOC Code', 'num_it_adopted']

X = create_factor_df('../data/db09/Skills.txt', 'LV')
X = X.merge(it_adoptions[['O*NET-SOC Code', 'num_it_adopted']], how = 'left', on = 'O*NET-SOC Code').drop('O*NET-SOC Code', axis = 1)
X.num_it_adopted = X.num_it_adopted.fillna(value = 0)
y = X.num_it_adopted
X = X.drop('num_it_adopted', axis = 1)
X = X[y.notnull()]
y = y[y.notnull()]

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
m, y_pred, name, rmse = optimize_gaussian(X_train, y_train, X_test, y_test, kernel_list)
score_model(y_pred, y_test)

Exponential RMSE: 13.8126093589
            
linear RMSE: 13.8126093217
            
ExpQuad RMSE: 13.8126093575
            
rbf RMSE: 13.8126093575
            
periodic_exponential RMSE: 13.812609338
            
    MODEL RESULTS:
    --------------

    Mean  |  6.6601  |
    RMSE  |  13.81261  |
    R2    |  0.30292  |
    


# Y = log(\# adopted emerging technologies)